In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cb = pd.read_csv("data/CB240302_Broadband.csv")
cbet = pd.read_csv("data/CB240302_Ethernet.csv")
msbbet = pd.read_csv("data/MS240302.csv")

In [3]:
cb.head()

,﻿Login,Route,Address,City,State,ZIP,County,Country,Requesting Company,User Company,Target Company,Request Date,Result,Product,Error Message
0,erik.sacchitella@crowncastle.com,Internal API,811 Louisiana St,Houston,TX,77002,NaN,USA,Crown Castle,NaN,Frontier Communications Corporation,2024-03-02T23:18:22.000Z,Fail,Ethernet - Switched,Pricing not found
1,erik.sacchitella@crowncastle.com,Internal API,811 Louisiana St,Houston,TX,77002,NaN,USA,Crown Castle,NaN,Frontier Communications Corporation,2024-03-02T23:18:22.000Z,Fail,Ethernet - Dedicated,Pricing not found
2,erik.sacchitella@crowncastle.com,Internal API,811 Louisiana St,Houston,TX,77002,NaN,USA,Crown Castle,NaN,Frontier Communications Corporation,2024-03-02T23:18:22.000Z,Fail,Ethernet - Switched w/COS,Pricing not found
3,bapostolico@spectrotel.com,Internal API,3085 Loma Vista Rd,Ventura,CA,93003,NaN,USA,Spectrotel,NaN,Frontier Communications Corporation,2024-03-02T21:22:04.000Z,Fail,Dedicated Internet - Managed,Pricing not found
4,bapostolico@spectrotel.com,Internal API,3085 Loma Vista Rd,Ventura,CA,93003,NaN,USA,Spectrotel,NaN,Frontier Communications Corporation,2024-03-02T21:22:04.000Z,Fail,Dedicated Internet,Pricing not found


In [4]:
msbbet.head()

,Request Date,User Company,Submitting User,RFQ#,Responsible,Product,Status,Address,City,State,ZIP,Old Address
0,3/2/2024 14:06,Fusion,James Abraham,5562126243,Unassigned,Broadband,Ready,153 HAZARD AVE,ENFIELD,CT,6082,"153 HAZARD AVE, ENFIELD, CT 06082"
1,3/2/2024 14:06,Fusion,James Abraham,2009298532,Unassigned,EIA,Ready,153 HAZARD AVE,ENFIELD,CT,6082,"153 HAZARD AVE, ENFIELD, CT 06082"
2,3/2/2024 14:05,Fusion,James Abraham,2647127163,Unassigned,Broadband,Ready,3645 DALLAS PKWY,PLANO,TX,75093,"3645 DALLAS PKWY, PLANO, TX 75093"
3,3/2/2024 14:05,Fusion,James Abraham,4606554875,Unassigned,EIA,Ready,3645 DALLAS PKWY,PLANO,TX,75093,"3645 DALLAS PKWY, PLANO, TX 75093"
4,3/2/2024 14:04,Fusion,James Abraham,3060623213,Unassigned,Broadband,Ready,330 MONROE AVE,ROCHESTER,NY,14607,"330 MONROE AVE, ROCHESTER, NY 14607"


In [5]:
msbbet['ZIP'] = msbbet['ZIP'].apply(lambda x : str(x).zfill(5))
msbbet1 = msbbet.copy()
msbbet1["Request Date"] = pd.to_datetime(msbbet["Request Date"], format="%m/%d/%Y %H:%M").dt.strftime("%Y-%m-%d")

In [6]:
msbbet2 = msbbet1[['Request Date', 'User Company','Product', 
                  'Address', 'City', 'State', 'ZIP']]
msbbet2.loc[(msbbet2['Product'] == 'EIA') | (msbbet2['Product'] == 'EPATH'), 'Product'] = 'Ethernet'

In [7]:
splits = msbbet2['User Company'].str.split(' RFQ')
cmpns = splits.str[0]
msbbet3 = msbbet2.copy()
msbbet3['User Company'] = cmpns
ms = msbbet3

In [8]:
cbbb1 = cb[(cb['Product'].isna())| (cb['Product'] == 'Broadband')| (cb['Product'] == 'Static IP')]
cbbb1['Product'] = 'Broadband'

cbet1 = cbet[(cbet['Product'].isna())| (cbet['Product'] == 'Ethernet - Switched w/COS') | 
            (cbet['Product'] == 'Ethernet - Switched w/COS2')| (cbet['Product'] == 'Ethernet - Switched')| 
            (cbet['Product'] == 'Dedicated Internet')]
cbet1['Product'] = 'Ethernet'

C:\Users\sss7371\AppData\Local\Temp\ipykernel_9996\473013913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbbb1['Product'] = 'Broadband'
C:\Users\sss7371\AppData\Local\Temp\ipykernel_9996\473013913.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbet1['Product'] = 'Ethernet'


In [9]:
cb = pd.concat([cbbb1, cbet1])

In [10]:
cb1 = cb.copy()
cb1['Request Date'] = cb["Request Date"].str[:10]

In [11]:
cb1["Request Date"] = pd.to_datetime(cb1["Request Date"], format="%Y-%m-%d")

In [12]:
cb2 = cb1[(cb1['Country'] == 'USA')] #use contains
cb2['User Company'] = cb2['User Company'].fillna(cb2['Requesting Company'])
cb3 = cb2[(cb2['User Company'] != 'Frontier Communications Corporation')]
cb3 = cb3.drop(['﻿Login', 'Error Message', 'Route', 'County', 'Country', 
                'Requesting Company', 'Target Company', 'Result'], axis = 1)
cb3['Address'] = cb3['Address'].str.upper()
cb3['City'] = cb3['City'].str.upper()

C:\Users\sss7371\AppData\Local\Temp\ipykernel_9996\4011195890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cb2['User Company'] = cb2['User Company'].fillna(cb2['Requesting Company'])


In [13]:
cbma = pd.concat([ms,cb3])
cbma["Request Date"] = pd.to_datetime(cbma["Request Date"], format="%Y-%m-%d")

In [14]:
states = ['AL', 'AZ', 'CA', 'CT', 'FL', 'GA', 'IA', 'IL' , 'IN', 'MI', 'MN', 'MS', 'NC', 
          'NE', 'NM', 'NV', 'NY', 'OH', 'PA', 'SC', 'TN', 'TX', 'UT', 'WI', 'WV' ]

In [15]:
cbma1 = cbma[cbma['State'].isin(states)].sort_values(axis = 0, by = ['Request Date'])

In [16]:
# rename output file
cbma1.to_csv('FinalQueries.csv', index=False)